In [1]:
import os

In [2]:
os.chdir('../')

In [5]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class Model_Trainer_Config:
    root_dir: Path
    model_path: Path
    data_path: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from src.txt_sm.constants import *
from src.txt_sm.utils.common import read_yaml,create_dirs


class Configuration_Manager:
    def __init__(self,config_path=CONFIG_PATH,params_path=PARAMS_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_dirs([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> Model_Trainer_Config:
        config = self.config
        
        create_dirs([config.root_dir])
        model_trainer_config = Model_Trainer_Config(root_dir=config.root_dir,
                                                    model_path=config.model_path,
                                                    data_path=config.data_path,
                                                    num_train_epochs=config.num_train_epochs,
                                                    warmup_steps=config.warmup_steps,
                                                    per_device_train_batch_size=config.per_device_train_batch_size,
                                                    per_device_eval_batch_size=config.per_device_eval_batch_size,
                                                    weight_decay=config.weight_decay,
                                                    logging_steps=config.logging_steps,
                                                    evaluation_strategy=config.evaluation_strategy,
                                                    eval_steps=config.eval_steps,
                                                    save_steps=config.save_steps,
                                                    gradient_accumulation_steps=config.gradient_accumulation_steps)
        return model_trainer_config
    

In [8]:
import urllib.request as request
import zipfile
from src.txt_sm.logging import logger
from src.txt_sm.utils.common import get_size
from transformers import set_seed,pipeline,Trainer,TrainingArguments
from datasets import load_dataset,load_from_disk,load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from transformers import DataCollatorForSeq2Seq

class Model_Trainer:
    def __init__(self,config:Model_Trainer_Config):
        self.config = config
    
    def initiate_model_trainer(self):
       device = 'cuda' if torch.cuda.is_available() else 'cpu'
       tokenizer = AutoTokenizer.from_pretrained(self.config.model_path)
       model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
       seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)
       
       dataset_pt = load_from_disk(self.config.data_path)
       
       training_args = TrainingArguments(
           output_dir=self.config.root_dir,
           num_train_epochs=self.config.num_train_epochs, 
           warmup_steps=self.config.warmup_steps, 
           per_device_train_batch_size=self.config.per_device_train_batch_size, 
           per_device_eval_batch_size=self.config.per_device_eval_batch_size, 
           weight_decay=self.config.weight_decay, 
           logging_steps=self.config.logging_steps, 
           evaluation_strategy=self.config.evaluation_strategy, 
           eval_steps=self.config.eval_steps, 
           save_steps=self.config.save_steps, 
           gradient_accumulation_steps=self.config.gradient_accumulation_steps
       )
       
       trainer = Trainer(model=model_pegasus,
                         args=training_args,
                         tokenizer=tokenizer,
                         data_collator=seq2seq_data_collator,
                         train_dataset=dataset_pt['train'],
                         eval_dataset=dataset_pt['validation'])
       
       trainer.train()
       
       model_pegasus.save_pretrained(os.path.join(self.config.root_dir,'pegasus-samsum-model'))
       
       tokenizer.save_pretrained(os.path.join(self.config.root_dir,'tokenizer'))